In [1]:
import tensorflow as tf
print(tf.test.gpu_device_name())

In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [14]:
# Load pre-trained MobileNet without the top classification layer
base_model = MobileNet(weights='imagenet', include_top=False)

C:\Users\pc\AppData\Local\Temp\ipykernel_31588\3831432045.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNet(weights='imagenet', include_top=False)


In [15]:
# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

In [16]:
# Add a new classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [17]:
# Create the full model
model = Model(inputs=base_model.input, outputs=predictions)


In [18]:
# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
# Define paths to your train, validation, and test data
train_data_dir = 'C:/Users/pc/Desktop/AUtism_database/Autism_database/AutismDataset/Train'
validation_data_dir = 'C:/Users/pc/Desktop/AUtism_database/Autism_database/AutismDataset/valid'
test_data_dir = 'C:/Users/pc/Desktop/AUtism_database/Autism_database/AutismDataset/Test'

In [20]:
# Define image dimensions
img_width, img_height = 224, 224

In [21]:
# Data augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1. / 255, # image pixels from [0,255] ====> [0,1]
    rotation_range=20, 
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest' # Startegy to fill empty pixels after shifting or rotation
)

In [22]:
# Data augmentation for validation and test data
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [23]:
# Data generators
batch_size = 32
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

Found 0 images belonging to 0 classes.
Found 100 images belonging to 2 classes.


In [24]:
# Train the model
with tf.device('/GPU:0'):
    
    model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // batch_size
)

ValueError: Asked to retrieve element 0, but the Sequence has length 0

In [ ]:
# Evaluate the model
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

loss, accuracy = model.evaluate(test_generator)
print("Test Accuracy:", accuracy)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Load the test data
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size, 
    shuffle=False,  
    class_mode='binary'
)

In [ ]:
# Predict on test data
predictions = model.predict(test_generator)
y_true = test_generator.classes
y_pred = predictions > 0.6  # threshold

In [ ]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

In [ ]:
# Plot confusion matrix
plt.figure(figsize=(8, 6))
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(2)
plt.xticks(tick_marks, ['Normal', 'Autistic'], rotation=45)
plt.yticks(tick_marks, ['Normal', 'Autistic'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')

# Text annotations
thresh = conf_matrix.max() / 2.
for i, j in ((i, j) for i in range(conf_matrix.shape[0]) for j in range(conf_matrix.shape[1])):
    plt.text(j, i, format(conf_matrix[i, j], 'd'),
             horizontalalignment="center",
             color="white" if conf_matrix[i, j] > thresh else "black")

plt.tight_layout()
plt.show()

In [ ]:
# Extract true negatives, false positives, false negatives, true positives
tn, fp, fn, tp = conf_matrix.ravel()
# Calculate sensitivity and specificity
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
print("Sensitivity (True Positive Rate):", sensitivity)
print("Specificity (True Negative Rate):", specificity)